In [3]:
import string
import re
from collections import deque

"""
Plan of attack:
- Use a list of deques as the data structure for the crates
- Parse the file by first reading the 'table' and turning it into the data structure
- Then parse the instructions into operations and execute them: move [multiplicity] from [source] to [target]
"""

with open('inputs/day5-input') as f:
    """ Find the table in the file """
    rows = 0
    line = f.readline()
    while not line.lstrip()[0].isdigit():
        rows += 1
        line = f.readline()

    """ Read the table into a data structure """
    col_offsets = [_.start() for _ in re.finditer(re.compile(r'\d'), line)]
    stop = f.tell()  # We've read far enough to capture the table
    f.seek(0)
    table_raw = f.readlines(stop)[:rows]
    table = [[row[_] for _ in col_offsets] for row in table_raw]

    queue_list = [deque() for _ in col_offsets]
    for r in table:
        for i, c in enumerate(r):
            if c != ' ':
                queue_list[i].appendleft(c)

    """ Follow the instructions """
    # I made this regex more brittle by encoding the instruction format directly into it. This ensures that any change to the source format will break the code, causing it to fail whatever tests would be written for it. Good interface design for something as flimsy as regex (or shell, etc.).
    regex = re.compile(r'^move\s*(?P<mult>\d+)\s*from\s*(?P<source>\d+)\s*to\s*(?P<target>\d+)$')

    f.seek(stop) # Return to the line after the table and its legend
    while True:
        line = f.readline()
        if len(line) != 0:  # Check for EOF
            instruction = re.match(regex, line.strip())
            if instruction:  # Check for a non-instruction line
                details = instruction.groupdict()
                for _ in range(int(details['mult'])):
                    queue_list[int(details['target']) - 1].append(queue_list[int(details['source']) - 1].pop())
        else:
            break

print(''.join([_.pop() for _ in queue_list]))

SHMSDGZVC
